In [14]:
import pandas as pd
import numpy as np

try: # If you are running this notebook on your local machine
  # Put the path to your CSV file here
  file_name = "./Customerbehavior_Final.csv"
  df = pd.read_csv(file_name, index_col=0, low_memory=False)
except : # If you are running this notebook on Google Colab
  from google.colab import drive
  drive.mount('/content/drive')
  # Put the path to your CSV file here
  file_name = "./drive/MyDrive/Data-IA/Customerbehavior_Final.csv"
  df = pd.read_csv(file_name, index_col=0, low_memory=False)

df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,StoreCode,BillCode,EnterTime,LeaveTime,BoxSize,MealType,OrderTime,MealSpending,BoxSpending,CustomerNumber,MaleNumber,FemaleNumber,Age,Membership
0,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Light Meal,2016-02-01 01:04:13,114,2857,6,3,3,30~39,0
1,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Project Package,2016-02-01 01:43:29,380,2857,6,3,3,30~39,0
2,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beer,2016-02-01 02:17:11,280,2857,6,3,3,30~39,0
3,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beverages (not including wine),2016-02-01 01:04:13,210,2857,6,3,3,30~39,0
4,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Fruit,2016-02-01 01:43:29,343,2857,6,3,3,30~39,0


# Data preparation

In [15]:
df.isnull().values.any()

False

## Apply complete store name

In [16]:
def store_name(value):
    if value == 107: return 'SOGO Store'
    elif value == 108: return 'Tao Yuan'
    elif value == 112: return 'Jia yi'
    elif value == 113: return 'Kaohsiung'
    elif value == 114: return 'Zhong Hua Xin dian'
    else: return 'Lin Sheng'

df['StoreName'] = df['StoreCode'].apply(store_name)
df.head(5)

,StoreCode,BillCode,EnterTime,LeaveTime,BoxSize,MealType,OrderTime,MealSpending,BoxSpending,CustomerNumber,MaleNumber,FemaleNumber,Age,Membership,StoreName
0,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Light Meal,2016-02-01 01:04:13,114,2857,6,3,3,30~39,0,Lin Sheng
1,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Project Package,2016-02-01 01:43:29,380,2857,6,3,3,30~39,0,Lin Sheng
2,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beer,2016-02-01 02:17:11,280,2857,6,3,3,30~39,0,Lin Sheng
3,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beverages (not including wine),2016-02-01 01:04:13,210,2857,6,3,3,30~39,0,Lin Sheng
4,107,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Fruit,2016-02-01 01:43:29,343,2857,6,3,3,30~39,0,Lin Sheng


## Create UniqueID

In [17]:
# Create "UniqueID" columns by concatenating StoreCode and BillCode
df["UniqueID"] = df['BillCode'].astype(str) + df['StoreCode'].astype(str)
df.drop(['StoreCode'], axis=1, inplace=True)
df.head()

,BillCode,EnterTime,LeaveTime,BoxSize,MealType,OrderTime,MealSpending,BoxSpending,CustomerNumber,MaleNumber,FemaleNumber,Age,Membership,StoreName,UniqueID
0,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Light Meal,2016-02-01 01:04:13,114,2857,6,3,3,30~39,0,Lin Sheng,1602010011107
1,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Project Package,2016-02-01 01:43:29,380,2857,6,3,3,30~39,0,Lin Sheng,1602010011107
2,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beer,2016-02-01 02:17:11,280,2857,6,3,3,30~39,0,Lin Sheng,1602010011107
3,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beverages (not including wine),2016-02-01 01:04:13,210,2857,6,3,3,30~39,0,Lin Sheng,1602010011107
4,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Fruit,2016-02-01 01:43:29,343,2857,6,3,3,30~39,0,Lin Sheng,1602010011107


## Change date string to date pandas object

In [18]:
# Create function to format the time stamp to date-time object
# def get_time(value):
#     temp = str.replace(value, "/", "-")
#     count1 = temp.count(":")
#     if count1 == 1: temp = temp + ":00"
#     return datetime.strptime(temp, "%Y-%m-%d %H:%M:%S")

# # Create new Time columns containing date-time object format
# df["TimeEnter"] = df["EnterTime"].apply(get_time)
# df["TimeLeave"] = df["LeaveTime"].apply(get_time)
# df["TimeOrder"] = df["OrderTime"].apply(get_time)

In [19]:
# The methods below are more efficient than 'apply' as they utilize pandas vectorized operations
# for string manipulation and datetime conversion,
# which are optimized for performance on entire columns.
df['EnterTime'] = df['EnterTime'].str.replace("/", "-").str.pad(19, side='right', fillchar='0')
df['LeaveTime'] = df['LeaveTime'].str.replace("/", "-").str.pad(19, side='right', fillchar='0')
df['OrderTime'] = df['OrderTime'].str.replace("/", "-").str.pad(19, side='right', fillchar='0')

df["EnterTime"] = pd.to_datetime(df["EnterTime"], format="%Y-%m-%d %H:%M:%S")
df["LeaveTime"] = pd.to_datetime(df["LeaveTime"], format="%Y-%m-%d %H:%M:%S")
df["OrderTime"] = pd.to_datetime(df["OrderTime"], format="%Y-%m-%d %H:%M:%S")

In [20]:
df["Duration(hours)"] = (df["LeaveTime"] - df["EnterTime"]) / np.timedelta64(1, 's') / 3600
df.head()

,BillCode,EnterTime,LeaveTime,BoxSize,MealType,OrderTime,MealSpending,BoxSpending,CustomerNumber,MaleNumber,FemaleNumber,Age,Membership,StoreName,UniqueID,Duration(hours)
0,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Light Meal,2016-02-01 01:04:13,114,2857,6,3,3,30~39,0,Lin Sheng,1602010011107,5.2
1,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Project Package,2016-02-01 01:43:29,380,2857,6,3,3,30~39,0,Lin Sheng,1602010011107,5.2
2,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beer,2016-02-01 02:17:11,280,2857,6,3,3,30~39,0,Lin Sheng,1602010011107,5.2
3,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Beverages (not including wine),2016-02-01 01:04:13,210,2857,6,3,3,30~39,0,Lin Sheng,1602010011107,5.2
4,1602010011,2016-02-01 00:49:00,2016-02-01 06:01:00,Small box,Fruit,2016-02-01 01:43:29,343,2857,6,3,3,30~39,0,Lin Sheng,1602010011107,5.2


## Caching

In [21]:
df.to_feather('./Customerbehavior_Final.feather')

In [22]:
df = pd.read_feather('./Customerbehavior_Final.feather')
df.shape, df.dtypes

((3169996, 16),
 BillCode                    int64
 EnterTime          datetime64[ns]
 LeaveTime          datetime64[ns]
 BoxSize                    object
 MealType                   object
 OrderTime          datetime64[ns]
 MealSpending                int64
 BoxSpending                 int64
 CustomerNumber              int64
 MaleNumber                  int64
 FemaleNumber                int64
 Age                        object
 Membership                  int64
 StoreName                  object
 UniqueID                   object
 Duration(hours)           float64
 dtype: object)

# Plotting

# IA
